# Imports

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
from tqdm import tqdm
import cv2
import torch
import matplotlib.pyplot as plt

from src.sam_segmentation.yolo import load_yolov8_detector, yolov8_detect
from src.sam_segmentation.sam import load_sam_predictor, sam_segmentation
from src.sam_segmentation.utils import unite_masks, masks_narrowing
from src.processing.watershed import perform_watershed

# Data

In [2]:
DATA_DIR = r"C:\Internship\ITMO_ML\data\raw_frames\Frames_bubbles\Bubbles_every_frame"

In [3]:
def filter_hidden_folders(folder):
    if folder[0] == '.':
        return False
    return True

In [4]:
folders_list = os.listdir(DATA_DIR)
folders_list = list(filter(filter_hidden_folders, folders_list))
folders_list

['F1_1_1_1_frames',
 'F1_1_1_2_frames',
 'F1_1_2_1_frames',
 'F1_1_2_2_frames',
 'F1_1_2_2_frames_masks',
 'F1_1_3_1_frames',
 'F1_1_3_2_frames',
 'F1_1_4_1_frames',
 'F1_1_4_2_frames',
 'F1_1_5_1_frames',
 'F1_1_5_2_frames',
 'F1_2_2_1_frames',
 'F1_2_2_2_frames',
 'F1_2_3_1_frames',
 'F1_2_3_2_frames']

In [5]:
def count_images(image_folder_path):
    images_num = len(os.listdir(image_folder_path))
    return images_num

def summarize_folders(data_dir, folders_list):
    folders_info = {}
    images_sum = 0
    
    for folder in folders_list:
        images_num = count_images(os.path.join(data_dir, folder))
        folders_info[folder] = images_num
        images_sum += images_num
        
    print(f"There is {images_sum} images:")
    for folder in folders_info.keys():
        print(f"\t{folder} folder has {folders_info[folder]} images")
    

In [6]:
summarize_folders(DATA_DIR, folders_list)

There is 21079 images:
	F1_1_1_1_frames folder has 1787 images
	F1_1_1_2_frames folder has 1798 images
	F1_1_2_1_frames folder has 1508 images
	F1_1_2_2_frames folder has 1491 images
	F1_1_2_2_frames_masks folder has 358 images
	F1_1_3_1_frames folder has 1479 images
	F1_1_3_2_frames folder has 1512 images
	F1_1_4_1_frames folder has 1797 images
	F1_1_4_2_frames folder has 1783 images
	F1_1_5_1_frames folder has 1502 images
	F1_1_5_2_frames folder has 1493 images
	F1_2_2_1_frames folder has 1474 images
	F1_2_2_2_frames folder has 1484 images
	F1_2_3_1_frames folder has 1480 images
	F1_2_3_2_frames folder has 133 images


# SAM+YOLO Labeling

Yolo detector:

In [7]:
custom_yolo_checkpoint_path = r"C:\Internship\ITMO_ML\CTCI\checkpoints\yolov8\yolov8s_on_mid_bubbles_69\run2_80_epochs\weights\best.pt"
detector = load_yolov8_detector(custom_yolo_checkpoint_path)

SAM:

In [8]:
sam_checkpoint = r"C:\Internship\ITMO_ML\CTCI\checkpoints\sam_checkpoints\sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda" if torch.cuda.is_available() else "cpu"

predictor = load_sam_predictor(checkpoint_path=sam_checkpoint, model_type=model_type, device=device)

## Labeling

In [9]:
def segment_images_from_folder(folder, target_length=800, narrowing=0.20, erode_iterations=1):
    masks_folder_name = folder + "_masks"
    dirlist = os.listdir(DATA_DIR)
    if masks_folder_name not in dirlist:
        os.mkdir(os.path.join(DATA_DIR, masks_folder_name))
    
    images_list = os.listdir(os.path.join(DATA_DIR, folder))
    masks_list = os.listdir(os.path.join(DATA_DIR, masks_folder_name))
    for image_name in tqdm(images_list):
        if image_name in masks_list:
            continue
        image = cv2.imread(os.path.join(DATA_DIR, folder,image_name))
        boxes = yolov8_detect(image=image, detector=detector, return_objects=False)
        masks_list = sam_segmentation(image=image, predictor=predictor, boxes=boxes, prompt_points=True, target_length=target_length)
        watershed = perform_watershed(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

        masks_united = []
        for masks in masks_list:
            masks_united.append(unite_masks(masks))
        masks_narrowed = masks_narrowing(masks_united, narrowing=narrowing)
        masks_sam = unite_masks(masks_narrowed)
        mask = unite_masks([masks_sam, watershed])
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        mask_final = cv2.erode(mask, kernel, iterations=erode_iterations)
        cv2.imwrite(filename=os.path.join(DATA_DIR, masks_folder_name, image_name), img=mask_final)

    print(f"Images from the folder {folder} were segmented!")

In [10]:
folder = folders_list[3]
folder

'F1_1_2_2_frames'

In [11]:
target_length=1024
narrowing = 0.20
erode_iterations = 1

In [24]:
# segment_images_from_folder(folder, target_length=target_length, narrowing=narrowing, erode_iterations=1)

  0%|          | 0/1491 [00:00<?, ?it/s]

0: 608x800 28 bubbles, 271.8ms
Speed: 4.0ms preprocess, 271.8ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)


  0%|          | 1/1491 [00:36<15:02:04, 36.33s/it]

C:\Users\PC\AppData\Local\Temp\ipykernel_12836\1588747203.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if image_name in masks_list:


0: 608x800 24 bubbles, 333.1ms
Speed: 3.0ms preprocess, 333.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)


...


0: 608x800 23 bubbles, 310.2ms
Speed: 4.0ms preprocess, 310.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)


 24%|██▍       | 358/1491 [4:11:40<11:38:48, 37.01s/it]


0: 608x800 22 bubbles, 284.3ms
Speed: 3.0ms preprocess, 284.3ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)


 24%|██▍       | 358/1491 [4:11:43<13:16:39, 42.19s/it]


KeyboardInterrupt: 

In [12]:
from src.sam_segmentation.sam import segment_images_from_folder

In [13]:
source_dir = os.path.join(DATA_DIR, folder)
output_dir = os.path.join(DATA_DIR, folder + "_masks")

processes_num = 2

In [ ]:
segment_images_from_folder(
    source_dir,
    output_dir,
    detector, predictor,
    target_length=target_length,
    narrowing=narrowing,
    erode_iterations=erode_iterations,
    processes_num=processes_num
)